In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import kraft

In [ ]:
na_pa = {}

na_se_li_ = {}

for pa in kraft.path.get_child("../raw/"):

    if pa[-3:] != ".md":

        continue

    na = pa.split("/")[-1][: -len(".md")]

    assert na == kraft.string.untitle(na)

    na_pa[na] = pa

    se_ = [".", "<", ">"]

    se_li_ = {se: [] for se in se_}

    with open(pa) as io:

        for li in io:

            li = li.strip()

            if li == "":

                continue

            if li[0] == "#":

                se = li[2]

                assert se_.pop(0) == se

                continue

            if li[0] == "(":

                # TODO: remove ()
                assert li[-2] == "."

                continue

            se_li_[se].append(li)

    na_se_li_[na] = se_li_

len(na_se_li_)

In [ ]:
for na, se_li_ in na_se_li_.items():

    for li in se_li_["<"]:

        assert kraft.string.untitle(li) not in na_se_li_, "{}: {}".format(na, li)

In [ ]:
import os
import shutil

In [ ]:
for se_li_ in na_se_li_.values():

    se_li_["md<"] = []

    se_li_[">md"] = []

In [ ]:
import pandas as pd

In [ ]:
fe_id = pd.read_csv("../feature.tsv", "\t", index_col=0, squeeze=True).to_dict()

In [ ]:
for na, se_li_ in na_se_li_.items():

    for li in list(se_li_[">"]):

        limi = kraft.string.untitle(li)

        if limi in na_se_li_:

            se_li_[">"].remove(li)

            assert li not in fe_id, li

            se_li_[">md"].append(li)

            na_se_li_[limi]["md<"].append(na)

In [ ]:
fe_ = []

for na, se_li_ in na_se_li_.items():

    for fe in se_li_["<"] + se_li_[">"]:

        if fe not in fe_id:

            fe_.append([na, fe])

assert len(fe_) == 0, fe_

In [ ]:
def prioritize(fe):

    if fe.endswith(".") or kraft.string.untitle(fe) in na_se_li_:

        return 0

    return fe_id[fe]

In [ ]:
# na_na2 = pd.read_csv("../rename.tsv", "\t", index_col=0, squeeze=True).to_dict()

na_na2 = {
    "A": "B",
}

In [ ]:
de = "\n\n"

In [ ]:
di = "../clean/"

for na, se_li_ in na_se_li_.items():

    pa = na_pa[na].replace("../raw/", di)

    kraft.path.make(pa, pr=False)

    with open(na_pa[na], "w") as io:

        ti_li_ = {
            ".": [],
            "<": [],
            ">": [],
        }

        for li in se_li_["."]:

            ti_li_["."].append(li)

        for li in se_li_["<"]:

            ti_li_["<"].append(na_na2.get(li, li))

        for li in se_li_[">"] + se_li_[">md"]:

            ti_li_[">"].append(na_na2.get(li, li))

        for ti, li_ in ti_li_.items():

            io.write("# {}".format(ti) + de)

            if len(li_) == 0:

                continue

            io.write(de.join(li_) + de)

In [ ]:
di = "../compiled/"

if os.path.isdir(di):

    shutil.rmtree(di)

for na, se_li_ in na_se_li_.items():

    pa = na_pa[na].replace("../raw/", di)

    kraft.path.make(pa, pr=False)

    with open(pa, "w") as io:

        ti_li_ = {
            "Comment": [],
            "Epidemiology": [],
            "Cause": [],
            "Symptom": [],
            "Sign": [],
            "Diagnostic": [],
            # "Treatment and Management": [],
        }

        for li in se_li_["."]:

            ti_li_["Comment"].append(li)

        for li in se_li_["<"]:

            if 4 <= fe_id[li] < 5:

                ti_li_["Epidemiology"].append(li)

        for li in se_li_["md<"]:

            ti_li_["Cause"].append(kraft.string.title(li))

        for li in se_li_[">"] + kraft.iterable.flatten(
            [na_se_li_[kraft.string.untitle(li)][">"] for li in se_li_[">md"]]
        ):

            if 1 <= fe_id[li] < 2:

                ti_li_["Symptom"].append(li)

            elif 6 <= fe_id[li] < 7:

                ti_li_["Sign"].append(li)

            elif 7 <= fe_id[li]:

                ti_li_["Diagnostic"].append(li)

        for ti, li_ in ti_li_.items():

            io.write("# {}".format(ti) + de)

            if len(li_) == 0:

                continue

            # TODO: check this independenly
            for li, n in pd.value_counts(li_).items():

                if 1 < n:

                    so_ = [na]

                    so_ += [
                        liri
                        for liri in se_li_[">md"]
                        if li in na_se_li_[kraft.string.untitle(liri)][">"]
                    ]

                    print("{}:\n\t{}\n".format(li, "\n\t".join(so_)))

            io.write(de.join(sorted(li_, key=prioritize)) + de)